# Objective

The goal of this project is to generate a segmentation mask for neurons that differentiates the soma and neurites from the background.
This primarily useful for automation features, such as morphological measurements, cell counting, etc.
We'll be doing this using a variant of U-net, which is a common choice for segmentation tasks in cellular biology. With the model defined,
we then train it on the synthetically generated data.

*Note: You can use the outline of this notebook to navigate the sections.*

# Import Dependencies

In [23]:
# Standard Library Imports
import logging
import time
import os
from pathlib import Path
from dataclasses import dataclass

# Third-Party Libraries
import numpy as np
import matplotlib. pyplot as plt 
import torch
from torch import nn, Tensor
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.v2.functional as FT
from torchvision.utils import make_grid
from torchvision.io import read_image
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm

# MLflow for Experiment Tracking and Model Management
import mlflow

# Configure Logging and Define Constants and Paths

In [24]:
# Configure the logging module with desired format and level
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(name)s | %(levelname)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Create a logger for this notebook
logger = logging.getLogger('segmentation-notebook')
logger.info("Logging configured successfully")

2025-06-09 03:59:53 | segmentation-notebook | INFO | Logging configured successfully


In [25]:
# Define global experiment and run names to be used throughout the notebook
PROJECT_NAME = 'neuron_segmentation'
RUN_NAME = "neuron_segmentation_main"
MODEL_NAME = "neuron_segmentation"
MLFLOW_EXPERIMENT = "Neuron Segmentation"
# Set up the paths
if Path('/phoenix').exists():
    # Use directories setup by Z by HP AI Studio
    ROOT = Path('/phoenix')
    TENSORLOGS_PATH = str(ROOT / 'tensorboard' / 'tensorlogs')
    MLFLOW_TRACKING_URI = str(ROOT / 'mlflow')
else:
    # Use these directories in all other cases.
    ROOT = Path('../data')
    TENSORLOGS_PATH = str(ROOT / 'tensorboard')
    MLFLOW_TRACKING_URI = str(ROOT / 'mlflow')

TRAIN_PATH = '../data/imagery/train'
VALIDATION_PATH = '../data/imagery/val'
BEST_MODEL_PATH = 'best_model.pth'
MODEL_PATH = 'neuron_segmentation.pt'

# Model Architecture and Implementation

The U-net model, in general, works by successively encoding features until the spatial resolution has been downsized by factor of 4 or 8.
After that, the model progressively up samples and concatenates features from prior encoding states. It can be thought of as an auto encoder
with skip connections. There are several variations, such as the addition of residual blocks or residual connections, but we will be using
the foundational version. The nice thing about U-net is that it is fully convolutional, so even if we train with a small spatial resolution,
we can use larger spatial resolutions during deployment.

Here is a diagram that illustrates what is happening visually.

<p align="center">
    <img src="../docs/unet.png" />
</p>

Many variants of U-net will go deeper, and many will also have batch normalization and activation functions along side the convolutions.
This diagram is a slight simplification in that it only includes the convolution, interpolation, and concatenation functions of the network.

## Encoder

In [26]:
class _Encoder(nn.Module):
    """
    Used for down sizing the input and encoding new features.
    This modules all have a stride of 2, a kernel size of 3, and a padding of 1.
    So any input signal that is passed to it will have an output whose spatial
    resolution is half of that of the input.
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        Initializes the encoder.

        :param in_channels: The number of input channels to the module.
        :param out_channels: The number of output channels to return by the module.
        """
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1, bias=False)
        self.norm = nn.BatchNorm2d(out_channels)

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor]:
        """
        Runs the forward pass of the module.

        :param x: The input signal to the module.

        :returns: The down sampled output of the module, pairsed with the original input signal.
                  The original input signal may be used later for skip connections to the decoders.
        """
        skip = x
        x = self.conv(x)
        x = self.norm(x)
        x = F.leaky_relu(x)
        return x, skip

## Decoder

In [ ]:
class _Decoder(nn.Module):
    """
    This module is used for up sampling the decoded feature maps by
    a factor of two and reconstructing detail. It also handles the
    concatenation of previous feature maps (in other words, handles
    the skip connections).
    """
    def __init__(self, in_channels: int, out_channels: int):
        """
        Initializes the decoder module.

        :param in_channels: The number of input channels for this module.
                            Note that this should include the number of channels
                            of the signal being concatenated.
        :param output_channels: The number of output channels to return from this module.
        """
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.norm = nn.BatchNorm2d(out_channels)

    def forward(self, x: Tensor, skip: Tensor) -> Tensor:
        """
        Runs the forward pass of this module.

        :param x: The input signal to upsample.
        :param skip: The skip connection to concatenate to the input signal.

        :returns: The output of the module.
        """
        shape = (skip.shape[2], skip.shape[3])
        x = F.interpolate(x, size=shape, mode='bilinear')
        x = torch.concat((x, skip), dim=1)
        x = self.conv(x)
        x = self.norm(x)
        x = F.leaky_relu(x)
        return x

## U-Net

In [ ]:
class UNet(nn.Module):
    """
    A class for building the architecture of the U-net model.
    """
    def __init__(self, input_features: int = 1):
        """
        Initializes the network.

        :param input_features: The number of channels used by the input image. For microscopy, this is generally single
                               channel, but it is sometimes 3 channels for widefield microscopy. For confocal microscopy,
                               this may be many channels, where each represents a thin slice of the specimen along the Z axis.
        """
        try:
            super().__init__()
            self.input_features = input_features
            self.enc1 = _Encoder(input_features, 16)
            self.enc2 = _Encoder(16, 128)
            self.enc3 = _Encoder(128, 256)
            self.dec3 = _Decoder(256 + 128, 128)
            self.dec2 = _Decoder(128 + 16, 16)
            self.dec1 = _Decoder(16 + 1, 3)
            logger.info("U-net initialization done successfully")
        except Exception as e:
            logger.error(f"Error initializing U-net: {e}")
            raise    

    def forward(self, x: Tensor) -> Tensor:
        """ 
        Implementation of the U-net logic, in which, the input passes through every step of the architecture.

        :param x: Input image from microscope.

        :returns: The segmentation mask of the input image, at the same spatial resolution.
        """
        try:
            x, skip0 = self.enc1(x)
            x, skip1 = self.enc2(x)
            x, skip2 = self.enc3(x)
            x = self.dec3(x, skip2)
            x = self.dec2(x, skip1)
            x = self.dec1(x, skip0)
            return x
        except Exception as e:
            logger.error(f"Error implementing U-net logic: {e}")
            raise 

    def smoke_test(self, res: tuple[int, int] = (512, 512), batch_size: int = 4):
        """
        This module is for verifying that the network correctly handles the input tensor.
        It primarily catches shape errors and input data type errors.

        :param res: The spatial resolution to test with.
        :param batch_size: The batch size to test with.
        """
        x = torch.randn((batch_size, self.input_features, res[0], res[1]))
        self.eval()
        try:
            y: Tensor = self.forward(x)
            assert(y.shape[0] == batch_size)
            assert(y.shape[1] == 3)
            assert(y.shape[2] == res[0])
            assert(y.shape[3] == res[1])
        except Exception as e:
            logger.error(f'Failed to run basic network test: {e}')
            raise

def check_unet():
    model = UNet()
    model.smoke_test()
check_unet()

2025-06-09 04:02:07 | segmentation-notebook | INFO | U-net initialization done successfully


# Data Loading and Preprocessing

In [5]:
@dataclass
class _Sample:
    """
    This class stores one training sample of the dataset.
    It consists of an input image and the target segmentation mask.
    The class will either store the path or the tensor. Initially,
    the path is only stored. Once the sample is used, the image is
    opened and the path is replaced with the image tensor. This is
    essentially lazy loading and speeds up training after the first
    epoch without having to wait for the whole dataset to be loaded
    before starting the training loop.
    """
    image: Path | Tensor
    target: Path | Tensor

class SegmentationDataset(Dataset):
    """
    This is a dataset for training the segmentation network.
    It reads a directory containing input images and segmentation
    mask images. The input images should be PNG files with 5 digits
    to indicate the index of the image (with zero padding). For
    example: "00000.png". Each one of this images should have a
    corresponding "{index:05}_mask.png", which is the target segmentation
    mask that the network should learn to reproduce. For example,
    the input image "00123.png" should have a corresponding "00123_mask.png".
    The segmentation mask should consist of three channels: red, green and blue.
    Red represents the soma, green represents a neurite, and blue represents
    the background.
    """
    def __init__(self, img_dir: Path, cache: bool):
        """
        Initialize the segmentation dataset.

        :param img_dir: Image directory containing input images and target segmentation masks.
        :param cache: Whether or not to cache the images in memory.
        """
        try:
            super().__init__()
            self.cache = cache
            self.samples: list[_Sample] = []
            for entry in img_dir.glob('*'):
                if 'mask' in entry.stem:
                    continue
                index = int(str(entry.stem))
                mask_entry = img_dir / f'{index:05}_mask.png'
                self.samples.append(_Sample(image=entry, target=mask_entry))

            logger.info(f"Segmentation dataset initialization done successfully (num_samples={len(self.samples)})")

        except Exception as e:
            logger.error(f"Error initializing segmentation dataset: {e}")
            raise

    def __getitem__(self, index: int) -> tuple[Tensor, Tensor]:
        """
        Gets an input image and the target segmentation mask.

        :index: The index of the sample in the dataset.

        :returns: An input image and a target segmentation mask.
        """
        try:
            sample = self.samples[index]
            if self.cache:
                if isinstance(sample.image, Path):
                    sample.image = read_image(str(sample.image)).float() * (1.0 / 255.0)
                if isinstance(sample.target, Path):
                    sample.target = read_image(str(sample.target)).float() * (1.0 / 255.0)
                image = sample.image
                target = sample.target
                return image, target
            else:
                image = read_image(str(sample.image)) * (1.0 / 255.0)
                target = read_image(str(sample.target)) * (1.0 / 255.0)
                return image, target
        except Exception as e:
            logger.error(f"Error getting sample: {e}")
            raise

    def __len__(self):
        """
        Get the number of input and output pairs.

        :returns: The number of input and output pairs.
        """
        return len(self.samples)


# Training and Validation

In [29]:
class TrainUNet(object):
    def __init__(self,
                 model: nn.Module,
                 optimizer: optim.Optimizer,
                 train_loader: DataLoader,
                 val_loader: DataLoader,
                 device):
        """
        Initialize the model training object.

        :model: U-net model.
        :criterion: Measures the error between the output and the real value.
        :optimizer : Adjusts model weights for loss minimization.
        :train_loader: Training batches.
        :val_loader: Validation batches.
        :n_epochs: Number of epochs for the training.
        :device: Where the model and data will be processed.
        """
        try:
            self.model = model
            self.optimizer = optimizer
            self.train_loader = train_loader
            self.val_loader = val_loader
            self.device = device
            self.log_dir = TENSORLOGS_PATH
            self.mlflow_exp_name = PROJECT_NAME
            self.tb_writer = torch.utils.tensorboard.SummaryWriter()
            self.best_metric = float('inf')
            self.best_epoch = -1

            logger.info('Training object initialization done successfully')
            
        except Exception as e:
            logger.error(f'  Error initializing training object: {e}')

    def train(self, num_epochs: int = 100):
        """
        Train the U-net model

        :param num_epochs: The number of epochs to train the model for.
        """
        try:
            writer = SummaryWriter(self.log_dir)
            mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
            mlflow.set_experiment(self.mlflow_exp_name)

            self.model = self.model.to(self.device)
            self.model.train()

            best_metrics: dict[str, float] | None = None

            for epoch in range(num_epochs):
                running_loss = 0.0
                count = 0
                loader = tqdm(self.train_loader)
                for i, (image, target) in enumerate(loader):
                    image: Tensor = image.to(self.device)
                    target: Tensor = target.to(self.device)

                    predicted = self.model(image)
                    loss: Tensor = F.cross_entropy(predicted, target)

                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()

                    running_loss += loss.item()
                    count += 1
                    loader.set_description(f'Train Loss: {running_loss / count}')

                self.log_images_to_tensorboard(writer, epoch)

                # Validate on the validation set
                metrics = self.validate()

                # Log metrics to Tensorboard
                for metric, value in metrics.items():
                    writer.add_scalar(metric, value, epoch)

                if running_loss < self.best_metric:
                    torch.save(self.model.state_dict(), BEST_MODEL_PATH)
                    best_metrics = metrics

            writer.close()

            with mlflow.start_run(run_name = RUN_NAME) as run:
                logger.info(run.info.run_id)
                if best_metrics is not None:
                    for metric_name, value in best_metrics.items():
                        mlflow.log_metric(metric_name, value)
                mlflow.register_model(model_uri = f'runs:/{run.info.run_id}/neuron-segmentation', name = MODEL_NAME)

            logger.info('Training done.')

        except Exception as e:
            logger.error(f'Error training model: {e}')

    def log_images_to_tensorboard(self, writer, epoch):
        """
        Log images on the tensorboard to monitor model training.

        :writer: Instance of the tensorboard SummaryWriter class, used to "write" in a format that the tensorboard can "read"
        :epoch: Epoch of each image
        """
        self.model.eval()
        try:
            with torch.no_grad():
                # Get a batch from the validation set
                image, target = next(iter(self.val_loader))
                image: Tensor = image.to(self.device)
                target: Tensor = target.to(self.device)

                # Forward pass
                predicted = self.model(image)
                grid = make_grid(torch.concat((image.expand(-1, 3, -1, -1), predicted, target), dim=0), n_rows=4)
                writer.add_image(f'Result', grid, epoch)

        except Exception as e:
            logger.error(f'Error logging on tensorboard: {e}')

    @staticmethod
    def __3_class_dice(pred: torch.Tensor, target: torch.Tensor, eps: float = 1e-6) -> Tensor:
        """
        Computes Dice coefficient for 3-class segmentation.

        :pred: Predicted class scores, shape [N, 3, H, W].
        :target: Ground truth, shape [N, 3, H, W].
        :eps: Smoothing term.

        :returns: Dice score per class.
        """
        pred_flat = pred.view(pred.shape[0], pred.shape[1], -1)
        target_flat = target.view(target.shape[0], target.shape[1], -1)

        intersection = (pred_flat * target_flat).sum(dim=2)
        union = pred_flat.sum(dim=2) + target_flat.sum(dim=2)

        dice = (2 * intersection + eps) / (union + eps)
        return dice.mean(dim=0)

    def validate(self) -> dict[str, float]:
        """
        Validates training by calculating some metrics that were defined.

        :returns: Metrics used to evaluate training.
        """
        try:
            self.model.eval()
            with torch.no_grad():
                val_loss = 0.0
                val_loop = tqdm(self.val_loader)
                count = 0
                dice_scores_sum = [0.0, 0.0, 0.0]
                for (image, target) in val_loop:
                    image: Tensor = image.to(self.device)
                    target: Tensor = target.to(self.device)
                    predicted = self.model(image)
                    loss: Tensor = F.cross_entropy(predicted, target)
                    dice_scores = TrainUNet.__3_class_dice(predicted, target)
                    dice_scores_sum[0] += dice_scores[0].item()
                    dice_scores_sum[1] += dice_scores[1].item()
                    dice_scores_sum[2] += dice_scores[2].item()
                    val_loss += loss.item()
                    count += 1
                    val_loop.set_description(f'Validation: Loss = {val_loss / count}')
                val_loop.close()
                dice_soma = dice_scores_sum[0] / count
                dice_neurites = dice_scores_sum[1] / count
                dice_background = dice_scores_sum[2] / count
                return {
                    'val_loss': val_loss / len(self.val_loader),
                    'dice_soma': dice_soma,
                    'dice_neurites': dice_neurites,
                    'dice_background': dice_background
                }
            
        except Exception as e:
            logger.error(f'Error validating training: {e}')
            raise


The evaluation metrics used are:

|          Metric | Description
|-----------------|------------
|      Train Loss | Evaluates the error in the data on which the model is learning. A decreasing loss means that the model is learning and improving
| Validation Loss | Evaluates the performance of the model on the validation set. It indicates how well the model can generalize to new data. In this case, it measures the error on the validation data. The lower the validation loss value, the better, meaning that the model is generalizing well.
|      Dice Score | This is a metric used to evaluate the performance of the network by computing the overlap between the predicted result and the ground truth.


In [30]:
class Args:
    train_dir = TRAIN_PATH
    val_dir = VALIDATION_PATH
    scale = 4
    batch_size = 4
    epochs = 1
    cache = False

args = Args()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logger.info(device)

mlflow.set_experiment(MLFLOW_EXPERIMENT)

# Load training dataset
train_dataset = SegmentationDataset(Path(args.train_dir), args.cache)

# Load validation dataset
val_dataset = SegmentationDataset(Path(args.val_dir), args.cache)

train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

model = UNet(input_features=1)
optimizer = optim.AdamW(model.parameters(), lr=0.0001)
trainer = TrainUNet(model, optimizer, train_loader, val_loader, device)
trainer.train(args.epochs)


2025-06-09 04:07:42 | segmentation-notebook | INFO | cuda
2025-06-09 04:07:43 | segmentation-notebook | INFO | Segmentation dataset initialization done successfully (num_samples=2240)
2025-06-09 04:07:43 | segmentation-notebook | INFO | Segmentation dataset initialization done successfully (num_samples=600)
2025-06-09 04:07:43 | segmentation-notebook | INFO | U-net initialization done successfully
2025-06-09 04:07:43 | segmentation-notebook | INFO | Training object initialization done successfully


  0%|          | 0/560 [00:00<?, ?it/s]

2025-06-09 04:10:13 | segmentation-notebook | ERROR | Error logging on tensorboard: make_grid() got an unexpected keyword argument 'nrows'


  0%|          | 0/150 [00:00<?, ?it/s]

2025-06-09 04:10:45 | segmentation-notebook | INFO | 8a55f959011949b8a19b9cb086d5d6bc
Registered model 'neuron_segmentation' already exists. Creating a new version of this model...
Created version '2' of model 'neuron_segmentation'.
2025-06-09 04:10:45 | segmentation-notebook | INFO | Training done.


# Inference

In [13]:
torch.save(model.state_dict(), MODEL_PATH)

In [16]:
model = UNet(input_features=1)
model.to(device)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

x_batch, y_batch = next(iter(val_loader))
x, y = x_batch[0], y_batch[0]
x = x.to(device).unsqueeze(dim=0)
y = y.unsqueeze(dim=0)
pred = F.softmax(model(x))
pred = pred.cpu()

# Convert tensors to numpy arrays 
y = y[0].numpy().transpose(1, 2, 0)
pred = pred[0].detach().numpy().transpose(1, 2, 0)

# Plot the images
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].imshow(y)
axes[0].set_title('Ground Truth')
axes[0].axis('off')

axes[1].imshow(pred)
axes[1].set_title('Predicted')
axes[1].axis('off')

2025-06-09 03:32:23 | segmentation-notebook | INFO | U-net initialization done successfully
/tmp/ipykernel_193/1247619192.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
 

TypeError: Module.load_state_dict() got an unexpected keyword argument 'weights_only'

Built with ❤️ using Z by HP AI Studio.